In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [3]:
#tail -n +2 train.csv | split -l 150000


In [4]:
train = pd.read_csv('train/xaa', float_precision='round_trip', header=None)
train.columns = ['acoustic_data','time_to_failure']

In [5]:
with pd.option_context('display.precision', 15):
    print(train.head())
    print(train.tail())

   acoustic_data  time_to_failure
0             12     1.4690999832
1              6     1.4690999821
2              8     1.4690999810
3              5     1.4690999799
4              8     1.4690999788
        acoustic_data  time_to_failure
149995              1     1.4307971903
149996              6     1.4307971892
149997              6     1.4307971881
149998              2     1.4307971870
149999              0     1.4307971859


In [6]:
all_features = ['acoustic_data', 'rolling_100']

In [7]:
features = all_features

In [8]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
acoustic_data      150000 non-null int64
time_to_failure    150000 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.3 MB


In [9]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [10]:
splits

['xzbvh',
 'xvv',
 'xzarc',
 'xzfad',
 'xum',
 'xhv',
 'xzdcy',
 'xiz',
 'xxn',
 'xzdlg',
 'xzanz',
 'xzeln',
 'xzagt',
 'xzdle',
 'xng',
 'xzbhs',
 'xzbko',
 'xzcsr',
 'xzckz',
 'xzdme',
 'xjd',
 'xtz',
 'xzaju',
 'xzavo',
 'xzbkq',
 'xyv',
 'xzbpt',
 'xzckn',
 'xzeog',
 'xzapz',
 'xzbdg',
 'xhx',
 'xzcrq',
 'xzbok',
 'xzejd',
 'xec',
 'xzcgi',
 'xhm',
 'xzbmd',
 'xcg',
 'xzeyt',
 'xzcdz',
 'xzbpr',
 'xln',
 'xzbzl',
 'xzcbv',
 'xzeia',
 'xzdoa',
 'xzffk',
 'xzbcd',
 'xzaae',
 'xzcik',
 'xzehd',
 'xzbck',
 'xzcnw',
 'xzdcn',
 'xzcvx',
 'xzcoy',
 'xzfff',
 'xzagf',
 'xjv',
 'xzdln',
 'xzciv',
 'xzahe',
 'xzbia',
 'xzclr',
 'xzdgh',
 'xzcir',
 'xzecg',
 'xzdfm',
 'xth',
 'xzbmj',
 'xzdwi',
 'xzera',
 'xzedf',
 'xhu',
 'xzasm',
 'xzehw',
 'xzboz',
 'xzffm',
 'xzbqb',
 'xzaua',
 'xzbhl',
 'xzdze',
 'xzabi',
 'xzcmm',
 'xzdha',
 'xzahq',
 'xzffq',
 'xzaoy',
 'xzdkz',
 'xzble',
 'xzdjh',
 'xjp',
 'xah',
 'xzevq',
 'xzcsc',
 'xzcmu',
 'xzbgf',
 'xzehx',
 'xhg',
 'xei',
 'xzbay',
 'xzdhm',
 '

In [133]:
splits_chunk = splits[0:1000]

In [134]:
training = []
ys = []

from sklearn.preprocessing import StandardScaler,MinMaxScaler 

for chunk in splits_chunk:
    path = 'train/%s' % (chunk)
    chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
    chunk_df.columns = ['acoustic_data','time_to_failure']
    chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    chunk_df[all_features] = StandardScaler().fit_transform(chunk_df[all_features])
    training.append(chunk_df[all_features].values)
    ys.append(chunk_df['time_to_failure'].values)

training = np.array(training)
ys = np.array(ys)

FileNotFoundError: [Errno 2] File b'train/[[ 3.55362567e+00  4.36201647e+01]\n [ 2.56833053e+00  3.75729216e+01]\n [ 9.26171960e-01  2.88380148e+01]\n [-8.80202468e-01  1.89272552e+01]\n [-1.86549761e+00  1.05619021e+01]\n [-2.19392932e+00  4.31308420e+00]\n [-3.17922447e+00 -1.87814094e+00]\n [-4.00030375e+00 -7.78140212e+00]\n [-3.34344032e+00 -1.14769396e+01]\n [-2.52236104e+00 -1.34254957e+01]\n [-1.37285004e+00 -1.37370204e+01]\n [-5.91231824e-02 -1.26527924e+01]\n [ 1.09038782e+00 -1.06499662e+01]\n [ 2.07568296e+00 -8.06936608e+00]\n [ 3.88205739e+00 -4.35463100e+00]\n [ 3.22519396e+00 -1.60817473e+00]\n [ 3.55362567e+00  1.05231583e+00]\n [ 2.56833053e+00  2.74528042e+00]\n [ 1.58303539e+00  3.62348629e+00]\n [ 4.33524389e-01  3.70835988e+00]\n [-8.80202468e-01  3.01724638e+00]\n [-3.87554897e-01  2.66383606e+00]\n [-8.80202468e-01  2.07823347e+00]\n [-1.70128175e+00  1.12148364e+00]\n [-2.52236104e+00 -1.61875740e-01]\n [-3.83608790e+00 -1.96674524e+00]\n [-4.16451961e+00 -3.7

In [48]:
ys.shape

(101, 150000)

In [14]:
training.shape

(101, 150000, 2)

In [15]:
#number of batches must cover a sequence length i.e. 100 * 1500 = 15000, batch size = 1500 

In [96]:
split_size = 500
new_training = []
new_ys = []
for i,sequence in enumerate(training):
    #split each sequence in split_size
    splits = np.array_split(sequence, split_size)
    #print(np.array(splits).shape)
    new_training.append(splits)
    splits_y = np.array_split(ys[i], split_size)
    new_ys.append(splits_y)

    
new_training = np.array(new_training)
new_ys = np.array(new_ys)

In [97]:
new_ys.shape

(101, 500, 300)

In [109]:
a = np.array([[[1,2,3], [4,5,6]], [[10,20,30], [40,50,60]]])
print(a.shape)
np.array(a[0])[:,-1]

(2, 2, 3)


array([3, 6])

In [112]:
a = np.array([[1,2,3], [4,5,6]])
a
a.reshape(-1)

array([1, 2, 3, 4, 5, 6])

In [89]:
np.array(new_ys[0])[:,-1].shape

(500,)

In [77]:
new_training.shape

(101, 500, 300, 2)

In [30]:
train = np.swapaxes(new_training,0,1)

In [118]:
train.shape

(50500, 300, 2)

In [98]:
targets = np.swapaxes(new_ys,0,1)

In [113]:
Y = []
for target in targets:
    Y.append(np.array(target[:,-1]))
    
Y = np.array(Y).reshape(-1)

In [114]:
Y.shape

(50500,)

In [23]:
a = np.array([[[[1],[2],[3],[4]],[[5],[6],[7],[8]],[[8],[9],[10],[11]]], [[[10],[20],[30],[40]],[[50],[60],[70],[80]],[[80],[90],[100],[110]]],\
              [[[1000],[2000],[3000],[4000]],[[5000],[6000],[7000],[8000]],[[8000],[9000],[10000],[11000]]] ])
print(a.shape)
a

(3, 3, 4, 1)


array([[[[    1],
         [    2],
         [    3],
         [    4]],

        [[    5],
         [    6],
         [    7],
         [    8]],

        [[    8],
         [    9],
         [   10],
         [   11]]],


       [[[   10],
         [   20],
         [   30],
         [   40]],

        [[   50],
         [   60],
         [   70],
         [   80]],

        [[   80],
         [   90],
         [  100],
         [  110]]],


       [[[ 1000],
         [ 2000],
         [ 3000],
         [ 4000]],

        [[ 5000],
         [ 6000],
         [ 7000],
         [ 8000]],

        [[ 8000],
         [ 9000],
         [10000],
         [11000]]]])

In [24]:
np.swapaxes(a,0,1).shape

(3, 3, 4, 1)

In [29]:
np.swapaxes(a,0,1).reshape(9,4,1)

(9, 4, 1)

In [115]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(train, Y, test_size=0.1, random_state=42)

In [124]:
train.shape

(50500, 300, 2)

In [132]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization

BATCH_SIZE=500

dropout=0.2

train

my_model = Sequential()

my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 3,\
                  stateful=True,batch_input_shape=(BATCH_SIZE,300,2),\
                  dropout=dropout,recurrent_dropout=dropout,
                  ))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0)
]

history = my_model.fit(train, Y, batch_size=BATCH_SIZE, epochs=100, shuffle=False,
                      #validation_data=(val_data,y_val), callbacks=callbacks
                      )


import matplotlib.pyplot as plt


#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (500, 3)                  72        
_________________________________________________________________
dense_7 (Dense)              (500, 1)                  4         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
50500/50500 [==============================] - 8s 164us/step - loss: 5.7566 - mean_absolute_error: 5.7566
Epoch 2/100
50500/50500 [==============================] - 8s 162us/step - loss: 4.8665 - mean_absolute_error: 4.8665
Epoch 3/100
50500/50500 [==============================] - 8s 156us/step - loss: 3.7537 - mean_absolute_error: 3.7537
Epoch 4/100
50500/50500 [==============================] - 9s 173us/step - loss: 3.4048 - mean_absolute_error: 3.4048
Epoch 5/100
50500/50500 [==============================] - 9s 183us/st

50500/50500 [==============================] - 9s 175us/step - loss: 3.2282 - mean_absolute_error: 3.2282
Epoch 65/100
50500/50500 [==============================] - 9s 173us/step - loss: 3.2279 - mean_absolute_error: 3.2279
Epoch 66/100
50500/50500 [==============================] - 9s 176us/step - loss: 3.2279 - mean_absolute_error: 3.2279
Epoch 67/100
50500/50500 [==============================] - 9s 178us/step - loss: 3.2284 - mean_absolute_error: 3.2284
Epoch 68/100
50500/50500 [==============================] - 9s 178us/step - loss: 3.2276 - mean_absolute_error: 3.2276
Epoch 69/100
50500/50500 [==============================] - 9s 170us/step - loss: 3.2278 - mean_absolute_error: 3.2278
Epoch 70/100
50500/50500 [==============================] - 9s 176us/step - loss: 3.2279 - mean_absolute_error: 3.2279
Epoch 71/100
50500/50500 [==============================] - 9s 178us/step - loss: 3.2276 - mean_absolute_error: 3.2276
Epoch 72/100
50500/50500 [==============================] - 9

KeyboardInterrupt: 

In [127]:
50500 / 500

101.0